# Apply on ABDDs demonstration 2

- this example yields a little bit more complex results, despite how small the inputs are
- it should also demonstrate how materialization introduces nodes into the resulting ABDD structure
- also notice the second example, which is a case of inefficient box order for canonization -- using L1 rule before H1 can be applied in many places, which results in a chain of nodes which could be more efficiently reduced by using H1 first

In [ ]:
import itertools

from apply.abdd_apply_main import abdd_apply
from apply.abdd import convert_ta_to_abdd
from apply.box_algebra.apply_tables import BooleanOperation
from apply.abdd_node_cache import ABDDNodeCacheClass
from apply.evaluation import compare_op_abdd

from formats.format_vtf import import_treeaut_from_vtf
from formats.render_dot import convert_to_dot

from apply.evaluation import compare_abdds_tas
from canonization.folding import ubda_folding
from canonization.normalization import ubda_normalize
from canonization.unfolding import remove_useless_transitions, ubda_unfolding
from helpers.string_manipulation import create_var_order_list
from helpers.utils import box_orders
from tree_automata.automaton import TTreeAut, iterate_edges
from tree_automata.functions.trimming import remove_useless_states

In [ ]:
ncache = ABDDNodeCacheClass()
varcount = 10
ta1 = import_treeaut_from_vtf("../tests/apply/ta-to-abdd-conversion/simple-input-1.vtf")
abdd1 = convert_ta_to_abdd(ta1, ncache, var_count=varcount)
convert_to_dot(abdd1)

In [ ]:
ta2 = import_treeaut_from_vtf("../tests/apply/ta-to-abdd-conversion/simple-input-2.vtf")
abdd2 = convert_ta_to_abdd(ta2, ncache, var_count=varcount)
convert_to_dot(abdd2)

In [ ]:
op = BooleanOperation.AND
result = abdd_apply(op, abdd1, abdd2, ncache, maxvar=varcount)
ncache.refresh_nodes()
print("abdd1 AND abdd2 = result:", compare_op_abdd(abdd1, abdd2, op, result))
convert_to_dot(result)

In [ ]:
convert_to_dot(abdd1)

In [ ]:
convert_to_dot(abdd2)

In [ ]:
op = BooleanOperation.OR
result = abdd_apply(op, abdd1, abdd2, ncache, maxvar=varcount)
ncache.refresh_nodes()
print("abdd1 OR abdd2 = result:", compare_op_abdd(abdd1, abdd2, op, result))
convert_to_dot(result)

In [ ]:
result.reformat_node_names()
result_ta = result.convert_to_treeaut_obj()
result_ta.reformat_states()
convert_to_dot(result_ta)

In [ ]:
unf = ubda_unfolding(result_ta, varcount+1)
convert_to_dot(unf)

In [ ]:
norm = ubda_normalize(unf, create_var_order_list("", varcount+1))
norm.reformat_states()
print(compare_abdds_tas(result, norm))
convert_to_dot(norm)

In [ ]:
fold = ubda_folding(norm, box_orders['full'], varcount+1)
fold.remove_self_loops()
fold = remove_useless_states(fold)
convert_to_dot(fold)

In [ ]:
abddx = convert_ta_to_abdd(fold, ncache=ABDDNodeCacheClass())
print(compare_abdds_tas(abddx, result))
convert_to_dot(abddx)